# HW3: Deep Q Networks

<!-- **Before doing this, be sure you review the `README.md` file from the homework!** -->

<!-- **HUGE NOTE**: the third deliverable asks you to run Pong for several thousand steps.
Please start early, as the evaluation portion of this assignment can take many hours.
We provide some timing benchmarks for machines in the third deliverable. -->

In this part of the assignment, you will implement the Deep Q-Network algorithm
(DQN) [1], along with the Double DQN extension [2]. 

**Rename your file with your student number and submit Axxxxx.ipynb to Luminus->files->Assignments->submissions->Assignment 3.**
<!-- At a high level, the `dqn.py`
code builds a TensorFlow computational graph in the class initialization method.
Then, to train, it iterates through environment steps and model updates. -->

## Review

Recall from lecture that the DQN algorithm performs the following optimization:

\begin{equation}
{\rm minimize}_{\theta} \;\; \mathbb{E}_{(s_t,a_t,r_t,s_{t+1})\sim D}
\left[ 
\Big(r_t + \gamma \max_{a \in \mathcal{A}} Q_{\theta^-}(s_{t+1},a) - Q_\theta(s_t,a_t)\Big)^2
\right]
\end{equation}

Here, $(s_t,a_t,r_t,s_{t+1})$ are batches of samples from the experience replay
buffer $D$, which is designed to store the past $N$ samples (where usually
$N=1,000,000$ for Atari benchmarks) so as to break correlation among the
training data used for updating $\theta$. In addition, we use $\theta$ to
represent the *current* or *online* network, whereas $\theta^-$
represents the *target* network. Both networks use the same architecture,
and we use $Q_\theta(s,a)$ or $Q_{\theta^-}(s,a)$ to denote which of the two
parameters is being applied to evaluate the state-action tuple $(s,a)$.

The target network starts off by getting matched to the current network, but
remains frozen (usually for thousands of steps) before getting updated again to
match the network. The process repeats throughout training, with the goal of
increasing the stability of the targets $r_t + \gamma \max_{a \in \mathcal{A}}
Q_{\theta^-}(s_{t+1},a)$. For more details, we recommend reading [1].

In *Double* DQN, the target value changes slightly. The new optimization
problem is:

\begin{equation}
{\rm minimize}_{\theta} \;\; \mathbb{E}_{(s_t,a_t,r_t,s_{t+1})\sim D}
\left[ 
\Big(r_t + \gamma Q_{\theta^-} \Big( s_{t+1}, {\rm argmax}_{a\in \mathcal{A}} Q_\theta(s_{t+1},a) \Big)  - Q_\theta(s_t,a_t)\Big)^2
\right]
\end{equation}

Notice the key difference in the target terms, which we denote $Y_t^{\rm DQN}$
and $Y_t^{\rm DoubleDQN}$ following the notation in [2]. Explicitly, we
have

\begin{equation}
Y_t^{\rm DQN} = r_t + \gamma \max_{a \in \mathcal{A}} Q_{\theta^-}(s_{t+1},a)
\end{equation}

in DQN, and

\begin{equation}
Y_t^{\rm DoubleDQN} = r_t + \gamma
Q_{\theta^-} \Big( s_{t+1}, {\rm argmax}_{a\in \mathcal{A}} Q_\theta(s_{t+1},a) \Big)
\end{equation}

in DDQN. Intuitively, DDQN helps to mitigate the issue of over-optimistic values
from DQN. In DQN, for a given state $s$, the Q-network takes the maximum over
quantities $Q_{\theta^-}(s,a_k)$ for all actions $a_k \in \mathcal{A}$. In DDQN,
we consider the same $Q_{\theta^-}(s,a_k)$ values, yet are not guaranteed to
take the maximum one because the action selection is done by a different
network.  For more details, we recommend reading [2]. The paper also
covers the Double Q-Learning algorithm, which was originally developed in the
tabular settings.  The Double *Deep Q-Network* algorithm, which you need to implement in this assignment, is the minimal extension of DQN towards Double Q-Learning.

# Noted that you need to implement Double DQN in this assignment.

[1]: [DQN](https://web.stanford.edu/class/psych209/Readings/MnihEtAlHassibis15NatureControlDeepRL.pdf), read the Nature 2015 paper, not the NIPS 2013 workshop paper

[2]: [DDQN](https://arxiv.org/abs/1509.06461) at AAAI, 2016.


# DDQN Implementation


**You will need to write code in `DQN.ipynb` file in three specific spots.
These are clearly indicated with the comments `START OF YOUR CODE` and
`END OF YOUR CODE`.** Put your code between those comments. Concretely, they correspond to:

- Environment Stepping
- Training by computing the Bellman Error.

**See the comments in `DQN.ipynb`, which provide detailed instructions. You
should not need to modify any other files for this part of the assigment**. 

Some advice: implementing DQN can be tricky, and it is sometimes difficult to know if your algorithm is working immediately due to the many samples necessary for deep reinforcement learning. Thus, we advice:

- Do not engage in (extensive) hyperparmeter tuning. We provide
hyperparameters for you in `DQN.ipynb` for CartPole environment that
have empirically worked for our solutions. If you have the time, you may find it
helpful to tune the learning rate for CartPole.

# Other Considerations

As in the first part on policy gradients, you should test your code by running Jupyter notebook cells. Make sure that you don't change the cells, as the graders need consistency in seeing the results.

In [ ]:
# watch first 12mins of Tutorial Week 5 (missed it)

!pip install gym==0.10.5

In [51]:
import random, time, gym, sys
import gym.spaces
from gym.wrappers.monitor import Monitor
import itertools
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torch.autograd as autograd
from collections import namedtuple
import pickle
import time
import uuid
import os
import sys
from collections import deque

# take Qteta- every 100 timestep

OptimizerSpec = namedtuple("OptimizerSpec", ["constructor", "kwargs", "lr_schedule"])

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Set Up Hyperparameters

In [52]:
env='CartPole-v0'
seed=3
num_steps=5e4
double_q=True

if seed is None:
    seed = random.randint(0, 9999)
print('random seed = {}'.format(seed))
exp_name = 'double-dqn'

logdir = exp_name+ '_' +env+ '_' +time.strftime("%d-%m-%Y_%H-%M-%S")
logdir = os.path.join('data_dqn', logdir)

def set_global_seeds(i):
    np.random.seed(i)
    random.seed(i)

env = gym.make(env)
set_global_seeds(seed)
env.seed(seed)
torch.manual_seed(seed)
expt_dir = os.path.join(logdir, "gym")
env = Monitor(env, expt_dir, force=True, video_callable=False)

random seed = 3
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [53]:
class DQN(nn.Module):

    def __init__(self, input_size, num_actions):
        super(DQN, self).__init__()
        self.dense1 = nn.Linear(input_size, 32)
        self.dense2 = nn.Linear(32, 32)
        self.dense3 = nn.Linear(32, num_actions)

    # Called with either one element to determine next action, or a batch
    # during optimization. Returns tensor([[left0exp,right0exp]...]).
    def forward(self, x):
        x = F.tanh(self.dense1(x))
        x = F.tanh(self.dense2(x))
        out = self.dense3(x)
        return out

## Set Up Exploration Strategy

In [54]:
def linear_interpolation(l, r, alpha):
    return l + alpha * (r - l)

class PiecewiseSchedule(object):

    def __init__(self, endpoints, interpolation=linear_interpolation, outside_value=None):
        """Piecewise schedule.
        
        Parameters
        ----------
        endpoints: [(int, int)]
            list of pairs `(time, value)` meanining that schedule should output
            `value` when `t==time`. All the values for time must be sorted in
            an increasing order. When t is between two times, e.g. `(time_a, value_a)`
            and `(time_b, value_b)`, such that `time_a <= t < time_b` then value outputs
            `interpolation(value_a, value_b, alpha)` where alpha is a fraction of
            time passed between `time_a` and `time_b` for time `t`.
        interpolation: lambda float, float, float: float
            a function that takes value to the left and to the right of t according
            to the `endpoints`. Alpha is the fraction of distance from left endpoint to
            right endpoint that t has covered. See linear_interpolation for example.
        outside_value: float
            if the value is requested outside of all the intervals sepecified in
            `endpoints` this value is returned. If None then AssertionError is
            raised when outside value is requested.
        """
        idxes = [e[0] for e in endpoints]
        assert idxes == sorted(idxes)
        self._interpolation = interpolation
        self._outside_value = outside_value
        self._endpoints = endpoints

    def value(self, t):
        """See Schedule.value"""
        for (l_t, l), (r_t, r) in zip(self._endpoints[:-1], self._endpoints[1:]):
            if l_t <= t and t < r_t:
                alpha = float(t - l_t) / (r_t - l_t)
                return self._interpolation(l, r, alpha)

        # t does not belong to any of the pieces, so doom.
        assert self._outside_value is not None
        return self._outside_value

exploration_schedule = PiecewiseSchedule([
        (0,   1.00), # At first, set as 1, it is totally random (exploratory stage)
        (1e4, 0.03), # At 10000th step, it becomes 0.03
        (2e4, 0.02), # At 20000th step, it becomes 0.02
    ], outside_value=0.02
)

## Set Up Replay Buffer

In [55]:
# ------------------------------------------------------------------------------
# REPLAY BUFFER
# ------------------------------------------------------------------------------

class BasicBuffer:
    
    def __init__(self, max_size):
        self.max_size = max_size
        self.buffer = deque(maxlen=max_size)

    def push(self, state, action, reward, next_state, done):
        experience = (state, action, np.array([reward]), next_state, done)
        self.buffer.append(experience)

    def sample(self, batch_size):
        state_batch = []
        action_batch = []
        reward_batch = []
        next_state_batch = []
        done_batch = []

        batch = random.sample(self.buffer, batch_size)

        for experience in batch:
            state, action, reward, next_state, done = experience
            state_batch.append(state)
            action_batch.append(action)
            reward_batch.append(reward)
            next_state_batch.append(next_state)
            done_batch.append(done)

        return (state_batch, action_batch, reward_batch, next_state_batch, done_batch)

    def __len__(self):
        return len(self.buffer)

def get_wrapper_by_name(env, classname):
    currentenv = env
    while True:
        if classname in currentenv.__class__.__name__:
            return currentenv
        elif isinstance(env, gym.Wrapper):
            currentenv = currentenv.env
        else:
            raise ValueError("Couldn't find wrapper named %s"%classname)

## Run Q-Learning Algorithm

## Noted you need to implement *Double DQN* in this assignment.

In [56]:
class QLearner(object):

    def __init__(self,
                 env,
                 q_func,
                 t_func,
                 optimizer_spec,
                 exploration,
                 replay_buffer_size,
                 batch_size,
                 gamma,
                 learning_starts,
                 target_update_freq,
                 grad_norm_clipping,
                 double_q=True,
                 max_steps=2e8,
                 cartpole=True):
        """Run Deep Q-learning algorithm.

        You can specify your own convnet using `q_func`.
        All schedules are w.r.t. total number of steps taken in the environment.

        Parameters
        ----------
        env: gym.Env
            gym environment to train on.
        q_func: function
            Model to use for computing the q function. 
        t_func: function
            Model to use for computing the target q function. 
        optimizer_spec: OptimizerSpec
            Specifying the constructor and kwargs, as well as learning rate schedule
            for the optimizer
        exploration: 
            exploration schedule
        replay_buffer_size: int
            How many memories to store in the replay buffer.
        batch_size: int
            How many transitions to sample each time experience is replayed.
        gamma: float
            Discount Factor
        learning_starts: int
            After how many environment steps to start replaying experiences
        target_update_freq: int
            How many experience replay rounds (not steps!) to perform between
            each update to the target Q network
        grad_norm_clipping: float or None
            If not None gradients' norms are clipped to this value.
        double_q: bool
            If True, use double Q-learning to compute target values. Otherwise, vanilla DQN.
            https://papers.nips.cc/paper/3964-double-q-learning.pdf
        max_steps: int
            Maximum number of training steps. The number of *frames* is 4x this
            quantity (modulo the initial random no-op steps).
        cartpole: bool
            If True, CartPole-v0.
        """
        assert type(env.observation_space) == gym.spaces.Box
        assert type(env.action_space)      == gym.spaces.Discrete
        self.q_func = q_func
        self.t_func = t_func
        self.max_steps = int(max_steps)
        self.target_update_freq = target_update_freq
        self.optimizer = optimizer_spec
        self.exploration = exploration
        self.batch_size = batch_size
        self.gamma = gamma
        self.learning_starts = learning_starts
        self.double_q = double_q
        self.cartpole = cartpole
        self.env = env
        self.replay_buffer = BasicBuffer(max_size=replay_buffer_size)

        self.input_shape = self.env.observation_space.shape # should be (4,)
        self.num_actions = self.env.action_space.n

        self.num_param_updates = 0
        self.mean_episode_reward      = -float('nan')
        self.std_episode_reward       = -float('nan')
        self.best_mean_episode_reward = -float('inf') 
        self.steps = []
        self.mean_rewards = []
        self.log_every_n_steps = 1000
        self.start_time = time.time()
        self.last_obs = self.env.reset()
        self.t = 0


    def step_env(self):
        """Step the env and store the transition.

        At this point, `self.last_obs` contains the latest observation that was
        recorded from the simulator. Here, your code
        needs to store this observation and outcome (reward, next observation,
        etc.) into the replay buffer while doing one step in the env simulator.

        At the end of this block of code, the simulator should have been
        advanced one step, the replay buffer should contain one more transition,
        and, `self.last_obs` must point to the new latest observation.

        Useful functions you'll need to call:

            self.env.step(action)
            self.replay_buffer.push(self.last_obs, action, reward, next_obs, done)

        This steps the environment forward one step. And:

            obs = self.env.reset()

        This resets the environment if you reached an episode boundary.  Call
        `self.env.reset()` to get a new observation if `done=True`. For CartPole, 
        this is guaranteed to start a new episode as they don't have a
        notion of 'ale.lives' in them.

        Don't forget to include epsilon greedy exploration!  
        """
        # ----------------------------------------------------------------------
        # START OF YOUR CODE
        # ----------------------------------------------------------------------
        # Hint: Don't forget to include epsilon greedy exploration!
        # if random.random() < self.exploration.value(self.t):
        #   ...
        # else:
        #   ...
        # ...
        # next_obs, _, _, _ = self.env.step(action)
        # Hint: Do not forget to update `self.last_obs` based on next observation (next_obs)
        # Hint: Call `self.env.reset()` to get a new observation and start a new episode if `done=True`.

        
        # like sample_action, store the trajectory into 
        # did not initialise yet, so you need to initialize a random action
        # epsilon value is between 0,1: get a number randomly if <value, else get it determinisitically
        # this allows us to switch around exploration and determinism
        
        if random.random() < self.exploration.value(self.t):
            # exploring
            action = random.randint(0, self.num_actions-1) 
            # [random.randint(0, self.num_actions-1) for i in range(self.batch_size)]
        else:
            # deterministic
            state_action_values = self.q_func(torch.FloatTensor(self.last_obs).to(device))
            action = torch.argmax(state_action_values, dim=-1).squeeze().detach().cpu().numpy()
        
        # step through env
        next_obs, reward, done, _ = self.env.step(action)
        # store experience
        self.replay_buffer.push(self.last_obs, action, reward, next_obs, done)
        # check if done/ update
        if done:
            self.last_obs = self.env.reset()
        else:
            # point to latest observation
            self.last_obs = next_obs

        # ----------------------------------------------------------------------
        # END OF YOUR CODE
        # ----------------------------------------------------------------------

    def update_model(self):
        """Perform experience replay and train the network.

        This is only done if the replay buffer contains enough samples for us to
        learn something useful -- until then, the model will not be initialized
        and random actions should be taken.  Training consists of four steps:
        
        3.a: Use the replay buffer to sample a batch of transitions. See the
        replay buffer code for function definitions. You may use this function:

          self.replay_buffer.sample(self.batch_size)

        3.b: Compute the total Bellman error in a batch. You can use MSE loss to
        achieve it.
        
        3.c: Perform a gradient step and update the network parameters to reduce
        total_error. 
        
        3.d: Periodically update the target network (self.t_func) every 
        `self.target_update_freq` steps.

        Hint: You can also use `param.grad.data.clamp` to clip gradient, where 
        `param in self.q_func.parameters()`.
        """
        if (self.t > self.learning_starts and \
            len(self.replay_buffer) > self.batch_size):

            # ------------------------------------------------------------------
            # START OF YOUR CODE
            # ------------------------------------------------------------------
            
            # use replay_buffer to sample the action pair and get the corresponding reward and next stage
            # get Q value network
            # update network using equation shown at the introduction
            
            # 3.a: Use the replay buffer to sample a batch of transitions
            current_obs, action, reward, next_obs, done = self.replay_buffer.sample(self.batch_size)
            reward = torch.FloatTensor(reward).squeeze().to(device)
            
            # 3.b: Compute the total Bellman error in a batch. You can use MSE loss. 
            current_state_action_values = self.q_func(torch.FloatTensor(current_obs).to(device))
            target_next_state_action_values = self.t_func(torch.FloatTensor(next_obs).to(device))
            
            current_state_action = torch.argmax(current_state_action_values, dim=-1)
            one_hot = F.one_hot(current_state_action, current_state_action_values.shape[1])
            pred_logits = torch.mul(one_hot, current_state_action_values)
            second_q_term = torch.sum(pred_logits, dim=1)
                
            if self.double_q:
                # DoubleDQN
                next_state_action_values = self.q_func(torch.FloatTensor(next_obs).to(device))
                
                next_state_action = torch.argmax(next_state_action_values, dim=-1)
                # next_state_action: torch.Size([100])

                one_hot = F.one_hot(next_state_action, target_next_state_action_values.shape[1])
                # one_hot: torch.Size([100, 2])
                
                first_q_term = torch.sum(torch.mul(one_hot, target_next_state_action_values), dim=1)
                # first_q_term: torch.Size([100])
                
                expected_state_action_values = reward + (self.gamma * first_q_term) - second_q_term
                
            else:
                # DQN
                
                next_state_action = torch.argmax(target_next_state_action_values, dim=-1)
                one_hot = F.one_hot(next_state_action, target_next_state_action_values.shape[1])
                first_q_term = torch.sum(torch.mul(one_hot, target_next_state_action_values), dim=1)
                
                expected_state_action_values = reward + (self.gamma * first_q_term) - second_q_term
            
            # 3.c: Perform a gradient step and update the network parameters to reduce total_error. 
            loss = torch.sum(torch.mul(expected_state_action_values,expected_state_action_values))
            
            self.optimizer.zero_grad()
            loss.backward()
            for param in self.q_func.parameters():
                param.grad.data.clamp_(-1, 1)
            self.optimizer.step()
            
            # 3.d: Periodically update the target network (self.t_func) every `self.target_update_freq` steps.
            if self.t%self.target_update_freq==0:
                # update target network
                for t,p in zip(target_net.parameters(), policy_net.parameters()):
                    t=p

            # ------------------------------------------------------------------
            # END OF YOUR CODE
            # ------------------------------------------------------------------
            self.num_param_updates += 1
        self.t += 1

    def log_progress(self):
        episode_rewards = get_wrapper_by_name(self.env, "Monitor").get_episode_rewards()

        if len(episode_rewards) > 0:
            self.mean_episode_reward = np.mean(episode_rewards[-10:])
            self.std_episode_reward  = np.std(episode_rewards[-10:])
        if len(episode_rewards) > 10:
            self.best_mean_episode_reward = \
                max(self.best_mean_episode_reward, self.mean_episode_reward)

        if self.t % self.log_every_n_steps == 0:
            self.steps.append(self.t)
            self.mean_rewards.append(self.mean_episode_reward)
            hours = (time.time() - self.start_time) / (60.*60.)
            print("Steps: ",                 self.t)
            print("Avg_Last_10_Episodes", self.mean_episode_reward)
            print("Std_Last_10_Episodes", self.std_episode_reward)
            print("Best_Avg_10_Episodes", self.best_mean_episode_reward)
            print("Num_Episodes",          len(episode_rewards))
            print("Exploration_Epsilon",   self.exploration.value(self.t))
            print("Elapsed_Time_Hours",    hours)


In [57]:
def dqn_learn(*args, **kwargs):
    alg = QLearner(*args, **kwargs)
    while True:
        # 
        alg.step_env()
        alg.update_model()
        alg.log_progress()
        if alg.t > alg.max_steps:
            print("\nt = {} exceeds max_steps = {}".format(alg.t, alg.max_steps))
            break
    return alg.steps, alg.mean_rewards

In [58]:
n_actions = env.action_space.n

policy_net = DQN(4, n_actions).to(device)
target_net = DQN(4, n_actions).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=5e-4)

steps, mean_rewards = dqn_learn(
    env=env,
    q_func=policy_net,
    t_func=target_net,
    optimizer_spec=optimizer,
    exploration=exploration_schedule,
    replay_buffer_size=50000,
    batch_size=100,
    gamma=0.99,
    learning_starts=1000,
    target_update_freq=500,
    grad_norm_clipping=10,
    double_q=double_q,
    max_steps=num_steps,
    cartpole=True
    )

# env.close()

Steps:  1000
Avg_Last_10_Episodes 27.3
Std_Last_10_Episodes 8.672369918309528
Best_Avg_10_Episodes 31.8
Num_Episodes 41
Exploration_Epsilon 0.903
Elapsed_Time_Hours 5.222201347351074e-05
Steps:  2000
Avg_Last_10_Episodes 18.1
Std_Last_10_Episodes 6.977822009767804
Best_Avg_10_Episodes 31.8
Num_Episodes 89
Exploration_Epsilon 0.806
Elapsed_Time_Hours 0.002623417642381456
Steps:  3000
Avg_Last_10_Episodes 14.7
Std_Last_10_Episodes 6.603786792439622
Best_Avg_10_Episodes 31.8
Num_Episodes 148
Exploration_Epsilon 0.7090000000000001
Elapsed_Time_Hours 0.005184189412328932
Steps:  4000
Avg_Last_10_Episodes 15.6
Std_Last_10_Episodes 5.370288632839022
Best_Avg_10_Episodes 31.8
Num_Episodes 209
Exploration_Epsilon 0.612
Elapsed_Time_Hours 0.007842087546984355
Steps:  5000
Avg_Last_10_Episodes 15.4
Std_Last_10_Episodes 6.887670143089026
Best_Avg_10_Episodes 31.8
Num_Episodes 274
Exploration_Epsilon 0.515
Elapsed_Time_Hours 0.010490513841311137
Steps:  6000
Avg_Last_10_Episodes 12.7
Std_Last_10_Ep

KeyboardInterrupt: 

```
Steps:  1000
Avg_Last_10_Episodes 27.3
Std_Last_10_Episodes 8.672369918309528
Best_Avg_10_Episodes 31.8
Num_Episodes 41
Exploration_Epsilon 0.903
Elapsed_Time_Hours 2.8939578268263076e-05
Steps:  2000
Avg_Last_10_Episodes 22.5
Std_Last_10_Episodes 9.31933474020544
Best_Avg_10_Episodes 31.8
Num_Episodes 86
Exploration_Epsilon 0.806
Elapsed_Time_Hours 0.0006945488188001845
Steps:  3000
Avg_Last_10_Episodes 34.9
Std_Last_10_Episodes 27.354889873658784
Best_Avg_10_Episodes 45.9
Num_Episodes 115
Exploration_Epsilon 0.7090000000000001
Elapsed_Time_Hours 0.0012762088245815702
Steps:  4000
Avg_Last_10_Episodes 77.3
Std_Last_10_Episodes 39.12045500757883
Best_Avg_10_Episodes 84.8
Num_Episodes 128
Exploration_Epsilon 0.612
Elapsed_Time_Hours 0.0021538283427556355
Steps:  5000
Avg_Last_10_Episodes 68.2
Std_Last_10_Episodes 55.26807396680294
Best_Avg_10_Episodes 89.4
Num_Episodes 142
Exploration_Epsilon 0.515
Elapsed_Time_Hours 0.003489295442899068
Steps:  6000
Avg_Last_10_Episodes 129.1
Std_Last_10_Episodes 63.62617385950533
Best_Avg_10_Episodes 133.5
Num_Episodes 149
Exploration_Epsilon 0.41800000000000004
Elapsed_Time_Hours 0.004759534796079
Steps:  7000
Avg_Last_10_Episodes 148.6
Std_Last_10_Episodes 53.8965676087077
Best_Avg_10_Episodes 161.2
Num_Episodes 154
Exploration_Epsilon 0.32100000000000006
Elapsed_Time_Hours 0.005507259832488166
Steps:  8000
Avg_Last_10_Episodes 167.4
Std_Last_10_Episodes 54.295856195477754
Best_Avg_10_Episodes 167.4
Num_Episodes 160
Exploration_Epsilon 0.22399999999999998
Elapsed_Time_Hours 0.006129175358348423
Steps:  9000
Avg_Last_10_Episodes 185.4
Std_Last_10_Episodes 43.8
Best_Avg_10_Episodes 185.4
Num_Episodes 165
Exploration_Epsilon 0.127
Elapsed_Time_Hours 0.006763347387313843
Steps:  10000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 170
Exploration_Epsilon 0.03
Elapsed_Time_Hours 0.007397124038802253
Steps:  11000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 175
Exploration_Epsilon 0.028999999999999998
Elapsed_Time_Hours 0.008027505212359958
Steps:  12000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 180
Exploration_Epsilon 0.028
Elapsed_Time_Hours 0.00865077641275194
Steps:  13000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 185
Exploration_Epsilon 0.027
Elapsed_Time_Hours 0.009281404217084248
Steps:  14000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 190
Exploration_Epsilon 0.026
Elapsed_Time_Hours 0.009908884763717651
Steps:  15000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 195
Exploration_Epsilon 0.025
Elapsed_Time_Hours 0.01054775701628791
Steps:  16000
Avg_Last_10_Episodes 198.6
Std_Last_10_Episodes 4.199999999999999
Best_Avg_10_Episodes 200.0
Num_Episodes 200
Exploration_Epsilon 0.024
Elapsed_Time_Hours 0.011194730665948656
Steps:  17000
Avg_Last_10_Episodes 197.0
Std_Last_10_Episodes 6.016643582596529
Best_Avg_10_Episodes 200.0
Num_Episodes 205
Exploration_Epsilon 0.023
Elapsed_Time_Hours 0.01183996147579617
Steps:  18000
Avg_Last_10_Episodes 198.4
Std_Last_10_Episodes 4.8
Best_Avg_10_Episodes 200.0
Num_Episodes 210
Exploration_Epsilon 0.022
Elapsed_Time_Hours 0.012484176092677646
Steps:  19000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 215
Exploration_Epsilon 0.020999999999999998
Elapsed_Time_Hours 0.013125873671637641
Steps:  20000
Avg_Last_10_Episodes 198.8
Std_Last_10_Episodes 3.6
Best_Avg_10_Episodes 200.0
Num_Episodes 220
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.013770669235123528
Steps:  21000
Avg_Last_10_Episodes 198.8
Std_Last_10_Episodes 3.6
Best_Avg_10_Episodes 200.0
Num_Episodes 225
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.014413673016760085
Steps:  22000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 230
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.01506945358382331
Steps:  23000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 235
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.01570997350745731
Steps:  24000
Avg_Last_10_Episodes 198.2
Std_Last_10_Episodes 3.8418745424597094
Best_Avg_10_Episodes 200.0
Num_Episodes 240
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.016356403628985088
Steps:  25000
Avg_Last_10_Episodes 198.2
Std_Last_10_Episodes 3.841874542459709
Best_Avg_10_Episodes 200.0
Num_Episodes 245
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.017002613014645045
Steps:  26000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 250
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.017655738393465677
Steps:  27000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 255
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.018303043709860908
Steps:  28000
Avg_Last_10_Episodes 197.7
Std_Last_10_Episodes 4.775981574503821
Best_Avg_10_Episodes 200.0
Num_Episodes 261
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.019076313508881462
Steps:  29000
Avg_Last_10_Episodes 197.7
Std_Last_10_Episodes 4.775981574503821
Best_Avg_10_Episodes 200.0
Num_Episodes 266
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.01986285706361135
Steps:  30000
Avg_Last_10_Episodes 199.2
Std_Last_10_Episodes 2.0880613017821097
Best_Avg_10_Episodes 200.0
Num_Episodes 271
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.021015156242582534
Steps:  31000
Avg_Last_10_Episodes 199.2
Std_Last_10_Episodes 2.0880613017821097
Best_Avg_10_Episodes 200.0
Num_Episodes 276
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.021655552718374466
Steps:  32000
Avg_Last_10_Episodes 199.0
Std_Last_10_Episodes 3.0
Best_Avg_10_Episodes 200.0
Num_Episodes 281
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.022295517126719157
Steps:  33000
Avg_Last_10_Episodes 196.9
Std_Last_10_Episodes 4.867237409455183
Best_Avg_10_Episodes 200.0
Num_Episodes 286
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.022941246827443442
Steps:  34000
Avg_Last_10_Episodes 197.5
Std_Last_10_Episodes 4.318564576337836
Best_Avg_10_Episodes 200.0
Num_Episodes 291
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.023599482046233283
Steps:  35000
Avg_Last_10_Episodes 199.6
Std_Last_10_Episodes 1.2
Best_Avg_10_Episodes 200.0
Num_Episodes 296
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.024243689444330002
Steps:  36000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 301
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.02489206227991316
Steps:  37000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 306
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.025543122953838773
Steps:  38000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 311
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.026197217636638216
Steps:  39000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 316
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.026856737136840822
Steps:  40000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 321
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.027519812915060254
Steps:  41000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 326
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.02819551044040256
Steps:  42000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 331
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.02886970493528578
Steps:  43000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 336
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.029555620816018847
Steps:  44000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 341
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.03021146323945787
Steps:  45000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 346
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.030879983107248942
Steps:  46000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 351
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.03154565023051368
Steps:  47000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 356
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.032307607531547544
Steps:  48000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 361
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.032972499198383756
Steps:  49000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 366
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.03363122271166907
Steps:  50000
Avg_Last_10_Episodes 200.0
Std_Last_10_Episodes 0.0
Best_Avg_10_Episodes 200.0
Num_Episodes 371
Exploration_Epsilon 0.02
Elapsed_Time_Hours 0.034312129219373065

t = 50001 exceeds max_steps = 50000
```

In [ ]:
import matplotlib.pyplot as plt
plt.plot(steps, mean_rewards)
plt.xlabel("Iterations")
plt.ylabel("Average Return")